In [1]:
import os
import sys
import gin
import numpy as np
import pandas as pd
from IPython.core.display import clear_output, display
from ariadne.graph_net.graph_utils.graph_prepare_utils import to_pandas_graph_from_df, get_pd_line_graph, \
    apply_nodes_restrictions, apply_edge_restriction, construct_output_graph
from ariadne.transformations import Compose, ConstraintsNormalize, ToCylindrical
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload

%autoreload 2

In [2]:
from prepare import parse

csv_params = {
"sep": '\s+',
"nrows": 15000,
"encoding": 'utf-8',
"names": ['event',  'x', 'y', 'z', 'station', 'track', 'px', 'py', 'pz', 'X0', 'Y0', 'Z0']
}
suff_df = ('_p', '_c')
gin.bind_parameter('get_pd_line_graph.restrictions_0',(-0.2, 0.2))
gin.bind_parameter('get_pd_line_graph.restrictions_1', (-0.28, 0.28))
gin.bind_parameter('get_pd_line_graph.suffix_c', '_c')
gin.bind_parameter('get_pd_line_graph.suffix_p','_p')
gin.bind_parameter('get_supernodes_df.suffix_c', '_c')
gin.bind_parameter('get_supernodes_df.suffix_p', '_p')
gin.bind_parameter('get_supernodes_df.axes', ['r', 'phi', 'z'])
gin.bind_parameter('apply_edge_restriction.edge_restriction', 0.093)
stations_constraints = {
    0: {'x': [-166.6, 166.6], 'y': [-166.6, 166.6], 'z': [-423.5, 423.5]},
    1: {'x': [-166.6, 166.6], 'y': [-166.6, 166.6], 'z': [-423.5, 423.5]},
    2: {'x': [-166.6, 166.6], 'y': [-166.6, 166.6], 'z': [-423.5, 423.5]},
}

In [3]:
ev_count = 1

events_count_str = f"0..{ev_count}"

events = parse("../data/new_data/1.txt",
               csv_params=csv_params, events_quantity=events_count_str,
               filter_mask=lambda df: df[df.track >= -1])
events = next(events)[0]
n_events = ev_count
events_to_analyze = events[events.event < n_events]



In [4]:
events_to_analyze


,event,x,y,z,station,track,px,py,pz,X0,Y0,Z0
0,0,-81.47063,3.708617,-117.40020,0,-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0,-81.47063,3.708617,53.01124,0,-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0,-28.08190,-76.567780,-238.43200,0,-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0,-28.08190,-76.567780,28.04412,0,-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0,-28.08190,-76.567780,156.29050,0,-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0,70.97662,-40.168860,-202.69470,0,-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0,70.97662,-40.168860,-106.63000,0,-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0,110.05800,61.458170,53.98859,1,-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0,-125.96900,4.654362,-79.59929,1,-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0,-42.52608,-118.665100,24.05098,1,-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [6]:
import torch
from torch_geometric.data import Data

In [7]:
n_stations = 3

for idx, event in events_to_analyze.groupby('event'):
    clear_output(wait=True)
    display("Event #%09d" % idx)

    transformer = Compose([
            ConstraintsNormalize(
                use_global_constraints=False,
                constraints=stations_constraints
            ),
            ToCylindrical(drop_old=True, cart_columns=('y', 'x'))
        ])
    try:
        event = transformer(event).reset_index()
    except AssertionError as err:
        print("ASS error %r" % err)
        continue

    def edge_list_from_ind(ind0, ind1):
        return np.concatenate(
            np.dstack(
                np.meshgrid(ind0, ind1, indexing='ij')
            ).transpose((0,2,1)),
            -1)

    indices = [None] * n_stations
    for station_idx, stat in event.groupby('station'):
        indices[station_idx] = stat.index.values
    edge_index = edge_list_from_ind(indices[0], indices[1])

    for station_idx in range(2, n_stations):
        edge_index = np.concatenate((edge_index,
                                     edge_list_from_ind(
                                         indices[station_idx-1],
                                         indices[station_idx]
                                     )), axis=1)
    node_features = event[['r','phi','z']].values
    y = event['track'].values
    y[y != -1] = 1
    y[y == -1] = 0
    assert (len(y) == len(node_features))
    geom_data = Data(x=torch.tensor(node_features),
                     edge_index=torch.tensor(edge_index),
                     y=torch.tensor(y))

'Event #000000000'

In [84]:
geom_data.is_directed()


True

In [8]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

n_features = 3
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(n_features, 128)
        self.conv2 = GCNConv(128, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.sigmoid(x)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
geom_data.x = geom_data.x.float()
data = geom_data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [13]:
model.train()
for epoch in range(1000):
    optimizer.zero_grad()
    out = model(data)
    loss = F.binary_cross_entropy((out.T)[0], data.y.float())
    loss.backward()
    optimizer.step()


In [14]:
model.eval()
pred = model(data)
correct = int(pred.eq(data.y).sum().item())
acc = correct / int(data.y.sum())
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.0000


In [15]:
pred

tensor([[0.0190],
        [0.3419],
        [0.0756],
        [0.2785],
        [0.0434],
        [0.0427],
        [0.1994],
        [0.3592],
        [0.3508],
        [0.3612],
        [0.3721],
        [0.3486],
        [0.3702],
        [0.4289],
        [0.4214],
        [0.4277],
        [0.4251],
        [0.4167],
        [0.6469],
        [0.3567],
        [0.4236],
        [0.7809],
        [0.3597],
        [0.4279],
        [0.9104],
        [0.3638],
        [0.4324],
        [0.8382],
        [0.3618],
        [0.4329]], grad_fn=<SigmoidBackward>)

In [164]:
data.y


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1])

In [20]:
out.T

tensor([[0.0248, 0.3126, 0.1272, 0.3231, 0.0295, 0.0489, 0.1815, 0.4388, 0.4259,
         0.4383, 0.4432, 0.4215, 0.4394, 0.4964, 0.4585, 0.4738, 0.4764, 0.4632,
         0.7564, 0.4397, 0.4425, 0.7650, 0.4313, 0.4830, 0.8987, 0.4410, 0.4752,
         0.8617, 0.4333, 0.4701]], grad_fn=<PermuteBackward>)

In [18]:
data.y.float().shape


torch.Size([30])